In [2]:
import numpy as np

In [3]:
# Función para calcular la norma \(p\) de una matriz \(x\) por filas
def lp_matrix(x: np.ndarray, p=1) -> float:
    if p == "inf":
        return np.max(np.abs(x), axis=1)
    return np.sum(np.abs(x)**p, axis=1)**(1/p)

# Función para calcular la norma \(p\) de un vector \(x\)
def lp_vector(x: np.ndarray, p=1) -> float:
    if p == "inf":
        return np.max(np.abs(x))
    return np.sum(np.abs(x)**p)**(1/p)

In [10]:
# Función que aproxima la norma \(p\) de una matriz \(A\) mediante muestreo Monte Carlo
def numeric_approach_lp_matrix_norm(matrix, p, samples):
    dimension = len(matrix)
    montecarlo = np.random.random((samples, dimension))
    
    # Normalizamos los vectores de Montecarlo
    montecarlo_norm = montecarlo / lp_matrix(montecarlo, p).reshape(-1, 1)

    # Calculamos la norma inducida
    norm_induced = lp_matrix(np.dot(matrix, montecarlo_norm.T).T, p)
    idx = np.argmax(norm_induced)

    # Comparamos con el valor real de la norma \(p\) de la matriz \(A\)
    if p != "inf":
        print("Numeric approach: ", lp_vector(np.dot(matrix, montecarlo_norm[idx, :]), p), " Vs. Real value: ", np.linalg.norm(matrix, ord=p))
    else:
        print("Numeric approach: ", lp_vector(np.dot(matrix, montecarlo_norm[idx, :]), p), " Vs. Real value: ", np.linalg.norm(matrix, ord=np.inf))
    
    return montecarlo_norm[idx, :]

In [11]:
# Función que aproxima la norma (p) de una matriz (A) mediante un enfoque iterativo
def iterative_numeric_approach_lp_matrix_norm(A, p, samples, max_iter=500):
    dimension = len(A)
    
    if p != "inf":
        real_val = np.linalg.norm(A, ord=p)
    else:
        real_val = np.linalg.norm(A, ord=np.inf)
    
    montecarlo = np.random.random((samples*10, dimension))

    # Normalizamos los vectores de Montecarlo
    montecarlo_norm = montecarlo / lp_matrix(montecarlo, p).reshape(-1, 1)

    # Calculamos la norma inducida inicial
    norm_induced = lp_matrix(np.dot(A, montecarlo_norm.T).T, p)
    idx = (-norm_induced).argsort()[:2]

    vector_a = montecarlo_norm[idx[0], :]
    vector_b = montecarlo_norm[idx[1], :]

    for j in range(max_iter):
        points_alpha = np.random.random((dimension, samples))
        points_beta = np.random.random((dimension, samples))

        # Combinamos los vectores alpha y beta
        combination = np.array([np.multiply(points_alpha[:, i], vector_a) + np.multiply(1-points_alpha[:, i], vector_b) for i in range(samples)]).reshape(-1, dimension)
        combination_norm = combination / lp_matrix(combination, p).reshape(-1, 1)

        # Calculamos la norma inducida con la combinación
        norm_induced = lp_matrix(np.dot(A, combination_norm.T), p)
        idx = (-norm_induced).argsort()[:2]

        # Si la diferencia es pequeña, consideramos que hemos convergido
        if np.abs(lp_vector(np.dot(A, combination_norm[idx[0], :]), p) - real_val) < 0.01:
            print(j)
            print("Iterative Numeric approach: ", lp_vector(np.dot(A, combination_norm[idx[0], :]), p), " Vs. Real value: ", real_val)
            return combination_norm[idx[0], :]
        else:
            if np.abs(lp_vector(np.dot(A, vector_a), p) - real_val) < np.abs(lp_vector(np.dot(A, combination_norm[idx[0], :]), p) - real_val):
                vector_a = combination_norm[idx[0], :]
                vector_b = combination_norm[idx[1], :]

    print(j)
    print("Iterative Numeric approach: ", lp_vector(np.dot(A, combination_norm[idx[0], :]), p), " Vs. Real value: ", real_val)
    return combination_norm[idx[0], :]


In [12]:
A = np.random.randint(-10,10,(5,5))
p = 2
print(A)

print("vector method 1: ", numeric_approach_lp_matrix_norm(A, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(A, p, 1000, 1000))

[[  5   1 -10   7   7]
 [-10  -1  -8   5 -10]
 [ -6   2   0   6   9]
 [  5  -2   2  -1   5]
 [ -7   4   7  -8   9]]
Numeric approach:  19.58030191972168  Vs. Real value:  21.02320500645222
vector method 1:  [0.22051183 0.15939952 0.56390183 0.0176827  0.77953086]
999
Iterative Numeric approach:  19.629161503374533  Vs. Real value:  21.02320500645222
Vector method iterative:  [0.18005283 0.24234921 0.43104469 0.03189246 0.84972418]


Usando el algoritmo del ítem anterior estime la norma inducida de
ordenes 1, 2, 3 y ∞ para la matríz de Hilbert de tamaño 10 × 10, definida por $H_{10} = [h_{ij}] = \frac{1}{i+j}$.

In [13]:
#Matriz de Hilbert de orden 10
H = np.array([[1/(i+j+1) for j in range(10)] for i in range(10)])

#Estimacion norma 1
p = 1

print("vector method 1: ", numeric_approach_lp_matrix_norm(H, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(H, p, 1000, 1000))

Numeric approach:  1.7612193295101093  Vs. Real value:  2.9289682539682538
vector method 1:  [0.28683157 0.01854333 0.22432508 0.22875937 0.00961482 0.07091147
 0.03255525 0.03629519 0.06866976 0.02349417]
999
Iterative Numeric approach:  1.8416297328468967  Vs. Real value:  2.9289682539682538
Vector method iterative:  [0.30549026 0.20489988 0.06454645 0.06220126 0.10061539 0.01186652
 0.11315769 0.06482484 0.06268405 0.00971366]


In [14]:
#Estimacion norma 2
p = 2

print("vector method 1: ", numeric_approach_lp_matrix_norm(H, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(H, p, 1000, 1000))

Numeric approach:  1.6719363221373684  Vs. Real value:  1.7519196702651776
vector method 1:  [0.58448069 0.36954789 0.39595526 0.2282977  0.11841353 0.39229218
 0.22052194 0.16243017 0.24793518 0.09228222]
999
Iterative Numeric approach:  1.710815233529692  Vs. Real value:  1.7519196702651776
Vector method iterative:  [0.59512714 0.40304965 0.38675427 0.27056278 0.22602555 0.31616831
 0.13508629 0.24372761 0.08064476 0.15933175]


In [15]:
#Estimacion norma infinita
p = "inf"

print("vector method 1: ", numeric_approach_lp_matrix_norm(H, p, 1000))

print("Vector method iterative: ", iterative_numeric_approach_lp_matrix_norm(H, p, 1000, 1000))

Numeric approach:  2.4862498190447204  Vs. Real value:  2.9289682539682538
vector method 1:  [1.         0.82938722 0.9954454  0.97843136 0.43633171 0.69330811
 0.52764932 0.41682243 0.90470819 0.64311134]
999
Iterative Numeric approach:  2.621417140275269  Vs. Real value:  2.9289682539682538
Vector method iterative:  [0.96820866 0.8667487  0.876524   0.8913758  0.75866457 0.96648442
 0.68210715 0.8267288  0.82095192 1.        ]
